In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quantas leis tem o Código de Defesa do consumidor?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 O Código de Defesa do Consumidor (CDC) é uma lei única, a Lei nº 8.078, de 11 de setembro de 1990. Essa lei é que estabelece as normas para a proteção e defesa dos direitos do consumidor em todo o território nacional.

Portanto, a resposta é: **Uma lei**.


In [6]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quantas leis tem o Código de Defesa do consumidor?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O Código de Defesa do Consumidor (CDC) é estabelecido pela Lei nº 8.078, de 11 de setembro de 1990. Esta lei estabelece normas de proteção e defesa do consumidor, sendo considerada de ordem pública e interesse social.


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Quantas leis código de defesa do consumidor']
Páginas utilizadas na resposta: senado.leg.br, www.gov.br, bvp.adv.br, normaslegais.com.br



In [8]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
################################################
# --- Agente 1: Classificador de perguntas --- #
################################################
def agente_classificador(topico, data_de_hoje):
    Classificador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que define se a pergunta e sobre o CDC",
        tools=[google_search],
        instruction=f"""

        Você é um classificador de perguntas. Sua única tarefa é determinar se a pergunta do usuário ('{topico}')
         é **sobre o Código de Defesa do Consumidor (CDC)** ou **não é sobre o Código de Defesa do Consumidor**.

        **REGRAS RÍGIDAS:**
        1. SE a pergunta ('{topico}') FOR claramente sobre o Código de Defesa do Consumidor, responda **SOMENTE** a palavra "CDC".
        2. SE a pergunta ('{topico}') NÃO FOR sobre o Código de Defesa do Consumidor, responda **SOMENTE** a palavra "OUTRO".
        3. NÃO inclua NENHUMA outra palavra, explicação, saudação ou pontuação na sua resposta, APENAS "CDC" ou "OUTRO".
        4. Não use ferramentas de busca para classificar. Baseie-se apenas na pergunta fornecida.

        Pergunta a classificar: '{topico}'

        Sua resposta deve ser APENAS "CDC" ou ""OUTRO".
        """
    )



    entrada_do_agente_classificador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    Pergunta_classificada = call_agent(agent=Classificador, message_text=entrada_do_agente_classificador)
    return Pergunta_classificada


In [13]:
##########################################
# --- Agente 2: Buscador de leis --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    pesquisador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction=f"""

        Você é um assistente de pesquisa. A sua tarefa é usar o
        site https://www.procon.df.gov.br/wp-content/uploads/2019/08/Codigo-do-consumidor-FINAL.pdf
        e a ferramenta de busca do google (google_search) para pesquisar artigos e materias para elaborar respostas sobre
        o Código de Defesa do Consumidor(CDC).
        Sempre leve em conta a matéria mais atual, verídica, legítima e confiável.
        Considere as informações do site https://www.procon.df.gov.br/wp-content/uploads/2019/08/Codigo-do-consumidor-FINAL.pdf
        como uma fonte importante para referência, mas use a busca para informações e atualizações recentes da Lei."""

    )



    entrada_do_agente_buscador = f"Tópico: {topico}\nData_de_hoje: {data_de_hoje}"
    leis_buscadas = call_agent(pesquisador, entrada_do_agente_buscador)
    return leis_buscadas


In [14]:
################################################
# --- Agente 3: Planejador de resposta --- #
################################################
def agente_planejador(topico, leis_buscadas):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description="Agente que planeja resosta",
        tools=[google_search],
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""

            Você e um especialista na lei de defessa do consumidor. Com base na lei de defesa do consumidor e pesquisa do buscador,
            você deve: usar as ferramentas de busca do google (google_search) e a Lei de defessa do consumidor
            para criar um plano de resposta com os pontos mais relevantes para o tópico pesquisado.
            Você pode também usar o (google_search) e o
            site https://www.procon.df.gov.br/wp-content/uploads/2019/08/Codigo-do-consumidor-FINAL.pdf
            para encontrar mais informações sobre o assunto. Sempre cite a lei do consumidor.
            Selecione e retorne as informações mais importantes e veridicas para elaborar uma resposta segura ao consumidor. """,

    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLeis_buscadas: {leis_buscadas}"
    # Executa o agente
    plano_de_resposta = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_resposta

In [15]:
##########################################
# --- Agente 4: Redator de Resposta --- #
##########################################
def agente_redator(topico, plano_de_resposta):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        description="Agente redator de Resposta para o consumidor",
        instruction="""
            Você é um Redator especialista em código de defesa do consumidor.
            **Primeiro cite em destaque, negrito, itálico e fonte maior, o artigo da lei do código de defesa do consumidor relacionada a pergunta feita pelo usuário.**
            Você escreve respostas para o agente da lei (Agente de IA), um agente de respostas assertivas, veridicas
            e legítimas sobre o Código de defesa do Consumidor.
            Utilize o tema fornecido pelo plano de resposta para elaborar um rascunho mais breve, simples e sucinto para o consumidor.
            Use linguagem simples, informal e direta.
            A resposta deve sempre levar em conta o Código de defesa do consumidor, ser verídica, informativa, com linguagem simples e incluir pelo menos 3 argumentos que o
            usuário comum possa usar para se defender e exigir que a lei seja cumprida.Sempre cite o artigo da lei do código de defesa do consumidor.
            """,

    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de resposta: {plano_de_resposta}"
    # Executa o agente
    resposta = call_agent(redator, entrada_do_agente_redator)
    return resposta

In [18]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print(" Iniciando o Sistema de Pesquisa do Código de Defesa do Consumidor ")

# --- Obter o Tópico do Usuário ---
topico = input(" Por favor, Qual sua dúvida sobre o Código de Defesa do consumidor: ")

# --- Importar a biblioteca time ---
import time

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Por favor, insira uma pergunta válida.")
else:
    print("\n")
    print(f"Só um momento, estamos pesquisando e elaborando a melhor resposta para você.")


    classificador_de_perguntas_raw = agente_classificador(topico, data_de_hoje)
    classificador_de_perguntas = classificador_de_perguntas_raw.strip() # <-- ADICIONADA ESTA LINHA PARA REMOVER ESPAÇOS E QUEBRAS DE LINHA

    print("\n--- Resultado do agente 1 (classificador) ---\n")
    display(to_markdown(classificador_de_perguntas_raw)) # Exibe o resultado original para debug, se necessário
    print(f"Resultado processado para comparação: '{classificador_de_perguntas}'") # Imprime o resultado processado para verificar
    print("--------------------------------------------")

    if classificador_de_perguntas == "OUTRO":
        # --- Bloco executado se a classificacao_de_pergunta for "OUTRO" (PARAR) ---
        print("\nDesculpe, minha especialidade é apenas o Código de Defesa do Consumidor. Não posso ajudar com tópicos fora desse assunto.")
        print("Processo finalizado: Tópico fora do escopo.") # Indica que o processo terminou aqui

    elif classificador_de_perguntas == "CDC":
        # --- Bloco executado se a classificação for "CDC" (CONTINUAR) ---
        print("\nSua pergunta é sobre CDC. Só um momento, estamos pesquisando e elaborando a melhor resposta para você.")

        # --- Chamar os agentes subsequentes SOMENTE AQUI DENTRO ---
        # Estão indentados sob o 'elif classificacao_da_pergunta == "CDC":'

        # Chamada para o Agente Buscador
        # Nota: Passe o 'topico' original e talvez a data para o buscador, não a classificação
        leis_buscadas = agente_buscador(topico, data_de_hoje) # Ajuste a entrada conforme a instrução do buscador
        print("\n--- Resultado do agente 1 (Buscador) ---\n")
        display(to_markdown(leis_buscadas))
        print("--------------------------------------------")

        time.sleep(2)

        plano_de_resposta = agente_planejador(topico, leis_buscadas)
        print("\n--- Resultado do agente 2 (Planejador) ---\n")
        display(to_markdown(plano_de_resposta))
        print("--------------------------------------------")

        time.sleep(2)

        rascunho_gerado = agente_redator(topico, plano_de_resposta)
        print("\n--- Resultado do agente 3 (Redator) ---\n")
        display(to_markdown(rascunho_gerado))
        print("--------------------------------------------")

    else: # Lida com resultados inesperados do classificador
        print(f"\nErro na classificação: Resultado inesperado '{classificador_de_perguntas}'. Não foi possível processar sua pergunta.")
        print("Processo finalizado: Erro na classificação.")

 Iniciando o Sistema de Pesquisa do Código de Defesa do Consumidor 
 Por favor, Qual sua dúvida sobre o Código de Defesa do consumidor: Recebi o produto com defeito, e agora?


Só um momento, estamos pesquisando e elaborando a melhor resposta para você.

--- Resultado do agente 1 (classificador) ---



> CDC


Resultado processado para comparação: 'CDC'
--------------------------------------------

Sua pergunta é sobre CDC. Só um momento, estamos pesquisando e elaborando a melhor resposta para você.

--- Resultado do agente 1 (Buscador) ---



> Olá! Entendo sua situação. Receber um produto com defeito é frustrante, mas o Código de Defesa do Consumidor (CDC) te protege nesses casos. Para te ajudar da melhor forma, preciso fornecer algumas informações sobre seus direitos e os passos que você pode seguir.
> 
> **Quais são seus direitos quando o produto vem com defeito?**
> 
> O CDC garante que você tem direito à reparação do vício (defeito) do produto. Isso significa que o fornecedor (quem te vendeu o produto) e o fabricante têm a responsabilidade de solucionar o problema.
> 
> **Prazos para solucionar o problema:**
> 
> *   **Produto não durável:** O prazo para reclamar do defeito é de 30 dias a partir da data da compra.
> *   **Produto durável:** O prazo para reclamar do defeito é de 90 dias a partir da data da compra.
> 
> **O que pode ser exigido do fornecedor/fabricante?**
> 
> Após a reclamação, o fornecedor/fabricante tem 30 dias para sanar o vício (resolver o problema). Caso não resolvam nesse prazo, você tem as seguintes opções, conforme o artigo 18 do CDC:
> 
> *   **Substituição do produto:** Trocar o produto defeituoso por outro em perfeitas condições.
> *   **Restituição do valor pago:** Receber o dinheiro de volta, com correção monetária.
> *   **Abatimento proporcional do preço:** Ficar com o produto e receber um desconto no valor, proporcional ao defeito.
> 
> **Como proceder?**
> 
> 1.  **Entre em contato com o fornecedor/fabricante:** Explique o problema detalhadamente e formalize a reclamação (guarde o protocolo, e-mails, etc.).
> 2.  **Guarde comprovantes:** Tenha em mãos a nota fiscal, o termo de garantia (se houver) e qualquer outro documento que comprove a compra e o defeito.
> 3.  **Se o problema não for resolvido em 30 dias:** Escolha uma das alternativas que mencionei acima (substituição, restituição ou abatimento) e comunique sua decisão ao fornecedor/fabricante.
> 4.  **Se não houver acordo:** Você pode registrar uma reclamação no Procon da sua cidade ou buscar auxílio judicial.
> 
> Para complementar essas informações e te dar mais segurança, farei uma pesquisa para verificar se houve alguma atualização recente sobre esse tema.
> 
> Com base nas pesquisas realizadas, compilei as seguintes informações adicionais e atualizações sobre seus direitos:
> 
> *   **Conceito de Vício e Defeito:** É importante saber que o CDC faz distinção entre "vício" e "defeito". Vício é um problema que afeta a qualidade ou quantidade do produto, enquanto defeito compromete a segurança, podendo causar danos ao consumidor (Fonte: Genyo; Advogados em Vinhedo).
> 
> *   **Produtos Essenciais:** Se o produto com defeito for considerado essencial (ex: geladeira, fogão), a troca ou solução do problema deve ser imediata, não sendo necessário aguardar o prazo de 30 dias (Fonte: Prefeitura de Rio Preto; Procon-ES).
> 
> *   **Responsabilidade Solidária:** Tanto o comerciante quanto o fabricante são responsáveis por solucionar o problema do produto com defeito (Fonte: PROCON).
> 
> *   **Indenização por Danos:** Além da troca, restituição ou abatimento, você pode ter direito a indenização por danos materiais e morais, ou até mesmo à saúde, caso o produto defeituoso cause algum prejuízo (Fonte: GOV.BR). O prazo para exigir essa reparação é de 5 anos, a partir do momento em que você toma conhecimento do problema e identifica o responsável.
> 
> *   **Direito de Arrependimento em Compras Online:** Se a compra foi feita pela internet ou telefone, você tem o direito de se arrepender em até 7 dias após o recebimento do produto, mesmo que não haja defeito (Fonte: Procon Pernambuco; Genyo).
> 
> **Recomendações:**
> 
> 1.  **Documente tudo:** Guarde todos os comprovantes de compra, protocolos de atendimento, e-mails trocados com o fornecedor/fabricante, etc.
> 2.  **Formalize a reclamação:** Faça a reclamação por escrito, detalhando o problema e suas expectativas de solução.
> 3.  **Procure o Procon:** Se não conseguir resolver diretamente com o fornecedor/fabricante, registre uma reclamação no Procon da sua cidade.
> 4.  **Considere a via judicial:** Se o problema persistir, procure um advogado para avaliar a possibilidade de ação judicial.
> 
> Espero que essas informações te ajudem a resolver essa situação da melhor forma possível!

--------------------------------------------

--- Resultado do agente 2 (Planejador) ---



> Compreendo a sua preocupação ao receber um produto com defeito. Para lidar com essa situação de forma eficaz, preparei um plano de resposta detalhado com base no Código de Defesa do Consumidor (CDC) e em pesquisas atualizadas.
> 
> **Plano de Resposta Detalhado:**
> 
> 1.  **Direitos do Consumidor:**
>     *   O CDC assegura o direito à reparação do defeito do produto, responsabilizando tanto o fornecedor quanto o fabricante pela solução do problema.
>     *   **Artigo 18 do CDC:** "Os fornecedores de produtos de consumo duráveis ou não duráveis respondem solidariamente pelos vícios de qualidade ou quantidade que os tornem impróprios ou inadequados ao consumo a que se destinam ou lhes diminuam o valor."
> 
> 2.  **Prazos para Reclamação:**
>     *   **Produtos não duráveis:** 30 dias a partir da data da compra.
>     *   **Produtos duráveis:** 90 dias a partir da data da compra.
>     *   **Artigo 26 do CDC:** "O direito de reclamar pelos vícios aparentes ou de fácil constatação caduca em: I - trinta dias, tratando-se de fornecimento de serviço e de produtos não duráveis; II - noventa dias, tratando-se de fornecimento de serviço e de produtos duráveis."
> 
> 3.  **Opções de Solução:**
>     *   Após a reclamação, o fornecedor/fabricante tem 30 dias para resolver o problema. Se não o fizerem, o consumidor pode escolher entre:
>         *   **Substituição do produto:** Troca por um produto novo e em perfeitas condições.
>         *   **Restituição do valor pago:** Reembolso integral do valor pago, com correção monetária.
>         *   **Abatimento proporcional do preço:** Desconto no valor do produto, mantendo-o com o defeito.
>     *   **Artigo 18, § 1º e § 3º do CDC:** Detalha as opções de substituição, restituição e abatimento proporcional.
> 
> 4.  **Procedimentos a Seguir:**
>     1.  **Contato com o Fornecedor/Fabricante:**
>         *   Formalize a reclamação detalhadamente, guardando todos os registros de comunicação (protocolos, e-mails, etc.).
>     2.  **Documentação:**
>         *   Reúna a nota fiscal, termo de garantia (se houver) e qualquer outro documento que comprove a compra e o defeito.
>     3.  **Aguarde o Prazo de 30 Dias:**
>         *   Se o problema não for resolvido nesse período, escolha uma das alternativas (substituição, restituição ou abatimento) e comunique sua decisão.
>     4.  **Registro no Procon:**
>         *   Se não houver acordo, registre uma reclamação no Procon da sua cidade.
>     5.  **Ação Judicial:**
>         *   Em último caso, procure auxílio judicial para buscar uma solução.
> 
> 5.  **Informações Adicionais e Atualizações:**
>     *   **Vício vs. Defeito:**
>         *   **Vício:** Afeta a qualidade ou quantidade do produto.
>         *   **Defeito:** Compromete a segurança e pode causar danos ao consumidor.
>     *   **Produtos Essenciais:**
>         *   A troca ou solução do problema deve ser imediata, sem a necessidade de aguardar os 30 dias.
>     *   **Responsabilidade Solidária:**
>         *   Tanto o comerciante quanto o fabricante são responsáveis por solucionar o problema.
>     *   **Indenização por Danos:**
>         *   Além da troca, restituição ou abatimento, você pode ter direito a indenização por danos materiais, morais e à saúde.
>         *   **Artigo 6º, VI, do CDC:** Garante a "efetiva prevenção e reparação de danos patrimoniais e morais, individuais, coletivos e difusos."
>     *   **Prazo para Reparação por Danos:**
>         *   5 anos a partir do conhecimento do problema e identificação do responsável.
>         *   **Artigo 27 do CDC:** "Prescreve em cinco anos a pretensão à reparação pelos danos causados por fato do produto ou do serviço prevista na Seção II deste Capítulo, iniciando-se a contagem do prazo a partir do conhecimento do dano e de sua autoria."
>     *   **Direito de Arrependimento em Compras Online:**
>         *   7 dias após o recebimento do produto, mesmo sem defeito.
>         *   **Artigo 49 do CDC:** "O consumidor pode desistir do contrato, no prazo de 7 dias a contar de sua assinatura ou do ato de recebimento do produto ou serviço, sempre que a contratação de fornecimento de produtos e serviços ocorrer fora do estabelecimento comercial, especialmente por telefone ou a domicílio."
> 
> 6.  **Recomendações Finais:**
>     *   **Documente Tudo:** Guarde todos os comprovantes e registros de comunicação.
>     *   **Formalize a Reclamação:** Faça a reclamação por escrito, detalhando o problema.
>     *   **Procure o Procon:** Registre uma reclamação no Procon se não conseguir resolver diretamente com o fornecedor/fabricante.
>     *   **Considere a Via Judicial:** Se o problema persistir, procure um advogado para avaliar a possibilidade de ação judicial.
> 
> Espero que este plano detalhado te ajude a resolver a situação de forma eficaz e a garantir seus direitos como consumidor.

--------------------------------------------

--- Resultado do agente 3 (Redator) ---



> ### ***"Art. 18. Os fornecedores de produtos de consumo duráveis ou não duráveis respondem solidariamente pelos vícios de qualidade ou quantidade que os tornem impróprios ou inadequados ao consumo a que se destinam ou lhes diminuam o valor, assim como por aqueles decorrentes da disparidade, com as indicações constantes do recipiente, da embalagem, rotulagem ou mensagem publicitária, respeitadas as variações decorrentes de sua natureza, podendo o consumidor exigir a substituição das partes viciadas."***
> 
> E aí, tudo bem? Que chato receber um produto com defeito, né? Mas fica tranquilo(a), a lei tá do seu lado! Olha só:
> 
> 1.  **Seus Direitos:** O Código de Defesa do Consumidor (CDC) diz que tanto a loja quanto o fabricante são responsáveis por resolver o problema. Artigo 18 do CDC.
> 
> 2.  **O Que Fazer:** Você tem até 90 dias pra reclamar de produtos duráveis (tipo eletrodomésticos) e 30 dias pra não duráveis (tipo comida). Depois de reclamar, eles têm 30 dias pra resolver. Se não resolverem, você pode escolher entre:
>     *   Trocar o produto por um novo;
>     *   Receber seu dinheiro de volta;
>     *   Ficar com o produto e receber um desconto.
> 
> 3.  **Como Se Defender:**
>     *   **Guarde tudo:** Nota fiscal, garantia, e-mails, tudo que possa provar sua compra e o defeito.
>     *   **Reclame por escrito:** Detalhe bem o problema e guarde uma cópia da reclamação.
>     *   **Procon:** Se a loja não resolver, corre pro Procon da sua cidade e faz uma reclamação.
> 
> Lembre-se, você não está sozinho(a) nessa! Seus direitos estão garantidos, e com essas dicas, você vai conseguir resolver essa situação rapidinho.

--------------------------------------------
